In [1]:
import pandas as pd
import numpy as np
import os
np.random.seed(42)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original', data_home=os.path.join(os.environ["HOME"],"datasets"))

In [3]:
enc = OneHotEncoder()
onehot_target = enc.fit_transform( mnist.target.reshape(-1,1)).toarray()

X_train, X_test, y_train, y_test = train_test_split(mnist.data,
                                                    onehot_target,
                                                    test_size=0.25, random_state=3)

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop
import keras
# keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)

batch_size = 128 #Number of images used in each optimization step
nb_classes = 10 #One class per digit
nb_epoch = 2 #Number of times the whole data is used to learn

dropout = 0.2
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(512, input_dim=784, init='uniform', activation='relu'))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dense(10, init='uniform', activation='softmax'))

# sgd = SGD(lr=.1, decay=1e-6, momentum=0.9, nesterov=True)
rms = RMSprop()

model.compile(loss='categorical_crossentropy',
              optimizer=rms,
              metrics=['accuracy'])

In [45]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.tr_loss = []
        self.tr_acc = []
        self.val_loss = []
        self.val_acc = []

    def on_batch_end(self, batch, logs={}):
        self.tr_loss.append(logs.get('loss'))
        self.tr_acc.append(logs.get('acc'))
        self.val_loss.append(logs.get('val_loss'))
        self.val_acc.append(logs.get('val_acc'))

In [46]:
history = LossHistory()
mo = model.fit(X_train, y_train,
          nb_epoch=nb_epoch,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          callbacks=[history])

score = model.evaluate(X_test, y_test, batch_size=batch_size)

Train on 52500 samples, validate on 17500 samples
Epoch 1/2
52500/52500 [==============================] - 27s - loss: 14.5370 - acc: 0.0981 - val_loss: 14.4529 - val_acc: 0.1033
Epoch 2/2
17280/17500 [============================>.] - ETA: 0s

In [47]:
mo.history

{'acc': [0.098095238095521925, 0.098095238097508752],
 'loss': [14.536986998276484, 14.53698701978411],
 'val_acc': [0.10331428571087974, 0.10331428571087974],
 'val_loss': [14.452865853881836, 14.452865853881836]}

In [32]:
history.

In [48]:
import  matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history.tr_acc, c="b:")
plt.plot(history.val_acc, c="r")

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x115ff77b8> (for post_execute):


ValueError: to_rgba: Invalid rgba arg "b:"
to_rgb: Invalid rgb arg "b:"
could not convert string to float: 'b:'

ValueError: to_rgba: Invalid rgba arg "b:"
to_rgb: Invalid rgb arg "b:"
could not convert string to float: 'b:'